In [70]:
import pandas as pd
import fastparquet

df = pd.read_parquet("preprocessed.parquet")

extracted_df = df[['cik', 'formType', 'filedAt']]

extracted_df


cik formType                    filedAt
0      1627554     10-K  2016-01-29T16:34:46-05:00
1       815093     10-K  2016-01-29T16:59:45-05:00
2      1598893     10-K  2016-01-29T18:26:19-05:00
3      1516887     10-K  2016-01-29T17:30:16-05:00
4      1084580     10-K  2016-01-29T16:59:43-05:00
...        ...      ...                        ...
58007  1901612     10-K  2023-12-05T16:11:23-05:00
58008   827876     10-K  2023-12-01T16:15:43-05:00
58009  1261654     10-K  2023-12-01T16:06:00-05:00
58010  1345865     10-K  2023-12-01T15:41:49-05:00
58011  1069533     10-K  2023-12-01T15:14:46-05:00

[58012 rows x 3 columns]

In [ ]:
def split_filedAt(s):
    parts = s.split("-", 2)
    year = parts[0]
    month = parts[1]
    rest = parts[2]
    
    if '-' in rest[1:]:
        day_time, tz = rest.rsplit("-", 1)
        tz = "-" + tz
    elif '+' in rest:
        day_time, tz = rest.rsplit("+", 1)
        tz = "+" + tz
    else:
        day_time = rest
        tz = None
    
    day = day_time.split("T")[0]
    date = month + "-" + day
    
    # Return a Series with explicit labels
    return pd.Series({'year': year, 'date': date, 'timezoneoffset': tz})


In [72]:
extracted_df.loc[:, ['year', 'date', 'timezoneoffset']] = extracted_df['filedAt'].apply(split_filedAt)

In [77]:
def check_data_and_save(full_df, year):
    
    df = full_df[full_df['year'] == year].copy() 
    
    original_length = len(df)
    
    with open(f'Data/{year}/{year}_cik_analysis.txt', "a") as f:
                f.write(f"{year} has originally {original_length} cik values.")
                f.write("\n")
                f.write(f'Number of values with timezone different to -5:00: {len((df[df['timezoneoffset'] != '-05:00']))}')
                f.write(f"\n")
                f.write(f'Analysis of Na:\n{df.isna().sum()}')
                f.write(f"\n")
                f.write(f'Number of duplicates: \n{df['cik'].duplicated().sum()}')
                f.write(f"\n")

    # Concat date
    df["full_date"] = pd.to_datetime(df["year"].astype(str) + "-" + df["date"], format="%Y-%m-%d")

    # Convert 'date' from MM-DD format to a sortable datetime format (e.g., month-day as a timestamp)
    df['date_sortable'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')

    # Drop duplicates based on 'cik', keeping the row with the latest 'date'
    filtered_df = df.sort_values('date_sortable').drop_duplicates(subset=['cik'], keep='last')
    
    result = filtered_df.loc[:, ["cik", "full_date"]]
    
    result["cik"] = result["cik"].astype(str).str.zfill(10)
    
    with open(f'Data/{year}/{year}_cik_analysis.txt', "a") as f:
        f.write(f"Due to filtering duplicates we lost {original_length - len(result)} cik values.")
        f.write(f"\n")
        f.write(f'Number of cik values without duplicates: {len(result)}')
    
    # Save the cleaned DataFrame
    result.to_parquet(f'Data/{year}/cik_{year}.parquet', engine='fastparquet')
    
    print(f"Data for {year} has been analyzed and saved into a parquet file.")

In [80]:
check_data_and_save(extracted_df, '2016')
check_data_and_save(extracted_df, '2017')
check_data_and_save(extracted_df, '2018')
check_data_and_save(extracted_df, '2019')
check_data_and_save(extracted_df, '2020')
check_data_and_save(extracted_df, '2021')
check_data_and_save(extracted_df, '2022')
check_data_and_save(extracted_df, '2023')


Data for 2018 has been analyzed and saved into a parquet file.
Data for 2019 has been analyzed and saved into a parquet file.
Data for 2020 has been analyzed and saved into a parquet file.
Data for 2021 has been analyzed and saved into a parquet file.
Data for 2022 has been analyzed and saved into a parquet file.
Data for 2023 has been analyzed and saved into a parquet file.
